# Evaluate Model Performance
Evalute model perfomance using the following metrics:
- Peak Signal to Noise Ratio (PSNR)
- Structural Similarity Index (SSIM)
- Spectral Angle Mapper (SAM)
- Spectral Correlation Coefficient (SCC)
- Mean Absolute Spectral Error (MASE)

In [ ]:
import os
import cv2
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import numpy as np

def compute_metrics(root_dir, upscale_method):
    hr_path = root_dir + "hr/"
    sr_path = root_dir + upscale_method + "/"
    scores = []
    img_id_list = os.listdir(hr_path)
    for img_id in img_id_list:
        print("Evaluating image: ", img_id)
        img_hr = np.load(os.path.join(hr_path, img_id))
        img_sr = np.load(os.path.join(sr_path, img_id))
        psnr_value = psnr(img_hr, img_sr, data_range=img_hr.max() - img_hr.min())
        ssim_value = ssim(img_hr, img_sr, data_range=img_hr.max() - img_hr.min(), channel_axis=2)
        scores.append({'Image_ID': img_id, 'PSNR': psnr_value, 'SSIM': ssim_value})

    return pd.DataFrame(scores)



# Choose model to evaluate
model = "HSI_x2_50000"
data_dir = "results/" + model + "/"

# Scores for SR and bicubic in a dataframe
df_sr = compute_metrics(root_dir=data_dir, upscale_method="sr")
df_bi = compute_metrics(root_dir=data_dir, upscale_method="bi_480")

# Merge the two dataframes
df_full = pd.merge(df_sr, df_bi, on='Image_ID', suffixes=('_SR', '_BI'))

# Save the mean scores to txt
with open(data_dir + model + "_means.txt", "w") as f:
    f.write("SR_PSNR: " + str(round(df_full['PSNR_SR'].mean(), 4)) + "\n")
    f.write("BI_PSNR: " + str(round(df_full['PSNR_BI'].mean(), 4)) + "\n")
    f.write("SR_SSIM: " + str(round(df_full['SSIM_SR'].mean(), 4)) + "\n")
    f.write("BI_SSIM: " + str(round(df_full['SSIM_BI'].mean(), 4)) + "\n")

# Print mean of SR and BI
print("Mean PSNR SR: ", round(df_full['PSNR_SR'].mean(), 4))
print("Mean PSNR BI: ", round(df_full['PSNR_BI'].mean(), 4))
print("Mean SSIM SR: ", round(df_full['SSIM_SR'].mean(), 4))
print("Mean SSIM BI: ", round(df_full['SSIM_BI'].mean(), 4))

# Save the dataframe to a csv file
# df_full.to_csv(data_dir + model + "_scores.csv", index=False)

Evaluating image:  spelt_m4.npy
Evaluating image:  wheatgrass_l4.npy
Evaluating image:  millet_l4.npy
Evaluating image:  flax_s4.npy
Evaluating image:  mix_m4.npy
Evaluating image:  pumpkin_l4.npy
Evaluating image:  rye_m4.npy
Evaluating image:  spelt_l4.npy
Evaluating image:  millet_m4.npy
Evaluating image:  pumpkin_s4.npy
Evaluating image:  mix_s4.npy
Evaluating image:  wheatgrass_m4.npy
Evaluating image:  rye_l4.npy
Evaluating image:  sunflower_l4.npy
Evaluating image:  sunflower_m4.npy
Evaluating image:  spelt_s4.npy
Evaluating image:  rye_s4.npy
Evaluating image:  sunflower_s4.npy
Evaluating image:  corn_s4.npy
Evaluating image:  flax_m4.npy
Evaluating image:  barley_s4.npy
Evaluating image:  flaxb_m4.npy
Evaluating image:  barley_l4.npy
Evaluating image:  mix_l4.npy
Evaluating image:  flaxb_l4.npy
Evaluating image:  buckwheat_s4.npy
Evaluating image:  buckwheat_m4.npy
Evaluating image:  millet_s4.npy
Evaluating image:  corn_l4.npy
Evaluating image:  corn_m4.npy
Evaluating image: 